In [19]:
# Import libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
# Set display options for better readability in output
#pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", '{:,.2f}'.format)

## **Data Overview**

### Load and explore all datasets

➤ Load all raw CSV files into individual DataFrames and store them in a dictionary for easier handling and quick access during exploration.

In [21]:
# Load all CSV files from the Brazilian E-Commerce dataset into separate DataFrames
data_path = "../data/brazilian/raw/"
files = os.listdir(data_path)

customers_raw = pd.read_csv(data_path + "olist_customers_dataset.csv")
geolocation_raw = pd.read_csv(data_path + "olist_geolocation_dataset.csv")
orders_raw = pd.read_csv(data_path + "olist_orders_dataset.csv")
items_raw = pd.read_csv(data_path + "olist_order_items_dataset.csv")
payments_raw = pd.read_csv(data_path + "olist_order_payments_dataset.csv")
reviews_raw = pd.read_csv(data_path + "olist_order_reviews_dataset.csv")
products_raw = pd.read_csv(data_path + "olist_products_dataset.csv")
sellers_raw = pd.read_csv(data_path + "olist_sellers_dataset.csv")
translation_raw = pd.read_csv(data_path + "product_category_name_translation.csv")

# Store all DataFrames in a dictionary for easier looping and inspection
dataframes_raw = {
    "customers": customers_raw,
    "geolocation": geolocation_raw,
    "orders": orders_raw,
    "items": items_raw,
    "payments": payments_raw,
    "reviews": reviews_raw,
    "products": products_raw,
    "sellers": sellers_raw,
    "translation": translation_raw,
}


The following tables are included in the Brazilian E-Commerce dataset:

- `customers`: customer information  
- `geolocation`: geographical coordinates by zip code prefix  
- `orders`: order details including status and timestamps  
- `items`: product-level details for each order  
- `payments`: payment methods, amounts and installment information 
- `reviews`: customer reviews and ratings  
- `products`: product attributes including category and dimensions
- `sellers`: seller information  
- `translation`: Portuguese-to-English product category mapping 

*Note: Original file names such as `olist_customers_dataset.csv` were renamed to simpler identifiers like `customers` for ease of use.*

➤  Summary of all tables using `.shape`, column names, and duplicate counts.

In [22]:
# Define a function to summarize a dictionary of DataFrames
def summarize_dataframes(df_dict=dataframes_raw):
    """
    Takes a dictionary of DataFrames and returns a summary DataFrame
        with the following information for each:
        - name: the key name from the dictionary
        - rows: number of rows in the DataFrame
        - columns: number of columns
        - column_names: a list of column names
        - duplicates: number of duplicated rows
    """
    summary = []

    # Loop over each DataFrame in the dictionary
    for name, df in df_dict.items():
        summary.append(
            {
                "name": name,
                "rows": df.shape[0],
                "columns": df.shape[1],
                "column_names": list(df.columns),
                "duplicates": df.duplicated().sum(),
            }
        )
    # Return a summary DataFrame
    return pd.DataFrame(summary)


# Call the function and display the summary of all loaded DataFrames
summarize_dataframes(dataframes_raw)

,name,rows,columns,column_names,duplicates
0,customers,99441,5,"[customer_id, customer_unique_id, customer_zip...",0
1,geolocation,1000163,5,"[geolocation_zip_code_prefix, geolocation_lat,...",261831
2,orders,99441,8,"[order_id, customer_id, order_status, order_pu...",0
3,items,112650,7,"[order_id, order_item_id, product_id, seller_i...",0
4,payments,103886,5,"[order_id, payment_sequential, payment_type, p...",0
5,reviews,99224,7,"[review_id, order_id, review_score, review_com...",0
6,products,32951,9,"[product_id, product_category_name, product_na...",0
7,sellers,3095,4,"[seller_id, seller_zip_code_prefix, seller_cit...",0
8,translation,71,2,"[product_category_name, product_category_name_...",0


➤  Quick sampling of 5 rows from each table for visual inspection.

In [23]:
# Display a random sample of 5 rows from each DataFrame for a quick visual inspection
for name, df in dataframes_raw.items():
    print(f'{name.capitalize()}:')
    display(df.sample(5))
    print("-"*130)

Customers:


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
12869,149ac0fdc3fc6ed91440c95c1a92736b,f1bee02ceae8a430506b798786f766b4,60170,fortaleza,CE
67812,3678034a0428898b73d8c6a2b353e769,7f71136093e2c5653179b2380e1cb00b,53433,paulista,PE
12260,294f90edf9d7f04ae89f4af0f2b64ed6,4f66517d8ac49edf81c5a6b15782d6bc,28735,quissama,RJ
79700,0505fd0a593f5eb5078c380b6761e43b,3da044832d726bc92b3d898b249aab53,95181,farroupilha,RS
4392,3e2fe6f0fa65fc4802dc9b29141f3a07,645f3682208dc4f2b7d0c935177551e2,36320,prados,MG


----------------------------------------------------------------------------------------------------------------------------------
Geolocation:


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
193531,7840,-23.32,-46.73,franco da rocha,SP
421455,21040,-22.86,-43.25,rio de janeiro,RJ
805837,75670,-17.98,-48.64,marzagão,GO
684250,44095,-12.29,-38.94,feira de santana,BA
469582,24020,-22.89,-43.12,niteroi,RJ


----------------------------------------------------------------------------------------------------------------------------------
Orders:


...

----------------------------------------------------------------------------------------------------------------------------------
Sellers:


,seller_id,seller_zip_code_prefix,seller_city,seller_state
880,d598f929fc44e1e38678e7f47250ec04,73801,formosa,GO
1796,41b86b552e54e3a7009596125aa8b167,2470,sao paulo,SP
2023,9d7a1d34a5052409006425275ba1c2b4,14403,franca,SP
204,6650fcccd8cd2f7e55ffa524f30c4c59,8490,sao paulo,SP
1484,e9c6969d40e6a3d2d0f90013165c2b98,3448,sao paulo,SP


----------------------------------------------------------------------------------------------------------------------------------
Translation:


,product_category_name,product_category_name_english
15,telefonia_fixa,fixed_telephony
63,portateis_casa_forno_e_cafe,small_appliances_home_oven_and_coffee
41,instrumentos_musicais,musical_instruments
69,fashion_roupa_infanto_juvenil,fashion_childrens_clothes
58,bebidas,drinks


----------------------------------------------------------------------------------------------------------------------------------


➤ Column-wise overview including dtypes, missing values, and unique counts.

In [24]:
# Quick overview of column properties (dtypes, missing values, uniques) for all DataFrames
def overview(df_dict=dataframes_raw):
    """
    Creates and displays a column-wise overview for each DataFrame in a dictionary.

    Parameters:
        df_dict (dict): A dictionary of DataFrames (e.g., {'orders': orders, ...})

    Displays:
        For each DataFrame:
            - Data type
            - Non-null count
            - Missing value count and percentage
            - Missing value percentage
            - Number of unique values
            - Unique values
    """
    for name, df in df_dict.items():
        print(f'{name.capitalize()}:')
        summary = pd.DataFrame(
                {
                    "dtype": df.dtypes,
                    "total": df.count(),
                    "missing_n": df.isna().sum(),
                    "missing_%": df.isna().mean() * 100,
                    "uniques_n": df.nunique(),
                    "uniques": [df[col].unique() for col in df.columns],
                }
        )
        display(summary)   
        print("-"*130)

overview(dataframes_raw)


Customers:


,dtype,total,missing_n,missing_%,uniques_n,uniques
customer_id,object,99441,0,0.00,99441,"[06b8999e2fba1a1fbc88172c00ba8bc7, 18955e83d33..."
customer_unique_id,object,99441,0,0.00,96096,"[861eff4711a542e4b93843c6dd7febb0, 290c77bc529..."
customer_zip_code_prefix,int64,99441,0,0.00,14994,"[14409, 9790, 1151, 8775, 13056, 89254, 4534, ..."
customer_city,object,99441,0,0.00,4119,"[franca, sao bernardo do campo, sao paulo, mog..."
customer_state,object,99441,0,0.00,27,"[SP, SC, MG, PR, RJ, RS, PA, GO, ES, BA, MA, M..."


----------------------------------------------------------------------------------------------------------------------------------
Geolocation:


,dtype,total,missing_n,missing_%,uniques_n,uniques
geolocation_zip_code_prefix,int64,1000163,0,0.00,19015,"[1037, 1046, 1041, 1035, 1012, 1047, 1013, 102..."
geolocation_lat,float64,1000163,0,0.00,717360,"[-23.54562128115268, -23.54608112703553, -23.5..."
geolocation_lng,float64,1000163,0,0.00,717613,"[-46.63929204800168, -46.64482029837157, -46.6..."
geolocation_city,object,1000163,0,0.00,8011,"[sao paulo, são paulo, sao bernardo do campo, ..."
geolocation_state,object,1000163,0,0.00,27,"[SP, RN, AC, RJ, ES, MG, BA, SE, PE, AL, PB, C..."


----------------------------------------------------------------------------------------------------------------------------------
Orders:


...

----------------------------------------------------------------------------------------------------------------------------------
Sellers:


,dtype,total,missing_n,missing_%,uniques_n,uniques
seller_id,object,3095,0,0.00,3095,"[3442f8959a84dea7ee197c632cb2df15, d1b65fc7deb..."
seller_zip_code_prefix,int64,3095,0,0.00,2246,"[13023, 13844, 20031, 4195, 12914, 20920, 5532..."
seller_city,object,3095,0,0.00,611,"[campinas, mogi guacu, rio de janeiro, sao pau..."
seller_state,object,3095,0,0.00,23,"[SP, RJ, PE, PR, GO, SC, BA, DF, RS, MG, RN, M..."


----------------------------------------------------------------------------------------------------------------------------------
Translation:


,dtype,total,missing_n,missing_%,uniques_n,uniques
product_category_name,object,71,0,0.00,71,"[beleza_saude, informatica_acessorios, automot..."
product_category_name_english,object,71,0,0.00,71,"[health_beauty, computers_accessories, auto, b..."


----------------------------------------------------------------------------------------------------------------------------------


➤ Quick statistical overview of all numeric columns in each raw table to spot any unusual values or patterns.

In [25]:
# Summarize basic statistics of all numeric columns for each DataFrame in the dictionary
def describe_numeric_columns(df_dict=dataframes_raw):
    """
    Displays a transposed summary of descriptive statistics (.describe().T)
    for all numeric columns in each DataFrame within the given dictionary.

    Parameters:
    df_dict (dict): A dictionary where keys are table names and values are pandas DataFrames.

    Notes:
    - If a DataFrame has no numeric columns, a message is printed instead.
    - The output includes a visual summary using display() for easier inspection in notebooks.
    """
    for name, df in df_dict.items():
        print(f"{name.capitalize()}:")
        numeric_df = df.select_dtypes(include="number")

        if numeric_df.empty:
            print("No numeric columns to describe.")
        else:
            display(numeric_df.describe().T)

        print("-" * 130)


describe_numeric_columns()

Customers:


,count,mean,std,min,25%,50%,75%,max
customer_zip_code_prefix,"99,441.00","35,137.47","29,797.94","1,003.00","11,347.00","24,416.00","58,900.00","99,990.00"


----------------------------------------------------------------------------------------------------------------------------------
Geolocation:


,count,mean,std,min,25%,50%,75%,max
geolocation_zip_code_prefix,"1,000,163.00","36,574.17","30,549.34","1,001.00","11,075.00","26,530.00","63,504.00","99,990.00"
geolocation_lat,"1,000,163.00",-21.18,5.72,-36.61,-23.60,-22.92,-19.98,45.07
geolocation_lng,"1,000,163.00",-46.39,4.27,-101.47,-48.57,-46.64,-43.77,121.11


----------------------------------------------------------------------------------------------------------------------------------
Orders:
No numeric columns to describe.
----------------------------------------------------------------------------------------------------------------------------------
Items:


...

----------------------------------------------------------------------------------------------------------------------------------
Products:


,count,mean,std,min,25%,50%,75%,max
product_name_lenght,"32,341.00",48.48,10.25,5.00,42.00,51.00,57.00,76.00
product_description_lenght,"32,341.00",771.50,635.12,4.00,339.00,595.00,972.00,"3,992.00"
product_photos_qty,"32,341.00",2.19,1.74,1.00,1.00,1.00,3.00,20.00
product_weight_g,"32,949.00","2,276.47","4,282.04",0.00,300.00,700.00,"1,900.00","40,425.00"
product_length_cm,"32,949.00",30.82,16.91,7.00,18.00,25.00,38.00,105.00
product_height_cm,"32,949.00",16.94,13.64,2.00,8.00,13.00,21.00,105.00
product_width_cm,"32,949.00",23.20,12.08,6.00,15.00,20.00,30.00,118.00


----------------------------------------------------------------------------------------------------------------------------------
Sellers:


,count,mean,std,min,25%,50%,75%,max
seller_zip_code_prefix,"3,095.00","32,291.06","32,713.45","1,001.00","7,093.50","14,940.00","64,552.50","99,730.00"


----------------------------------------------------------------------------------------------------------------------------------
Translation:
No numeric columns to describe.
----------------------------------------------------------------------------------------------------------------------------------


In [26]:
# Generate dbdiagram.io-compatible table definitions from DataFrames
def generate_er_schema(df_dict=dataframes_raw):
    """
    Generate table definitions in dbdiagram.io format from a dictionary of DataFrames.

    For each DataFrame:
    - Converts pandas dtypes to SQL-style types (int, varchar, decimal, timestamp, etc.)
    - Outputs formatted table definitions ready to paste into dbdiagram.io

    Parameters:
    df_dict (dict): A dictionary where keys are table names and values are pandas DataFrames.

    Returns:
    None (prints output to console)
    """

    dtype_map = {
        "int64": "int",
        "float64": "decimal",
        "object": "varchar",
        "bool": "boolean",
        "datetime64[ns]": "timestamp",
    }

    for name, df in df_dict.items():
        print(f"Table {name} {{")
        for col in df.columns:
            dtype = str(df[col].dtype)
            sql_type = dtype_map.get(dtype, "varchar")  # fallback to varchar if unknown
            print(f"  {col} {sql_type}")
        print("}\n")

generate_er_schema(dataframes_raw)

Table customers {
  customer_id varchar
  customer_unique_id varchar
  customer_zip_code_prefix int
  customer_city varchar
  customer_state varchar
}

Table geolocation {
  geolocation_zip_code_prefix int
  geolocation_lat decimal
  geolocation_lng decimal
  geolocation_city varchar
  geolocation_state varchar
}

Table orders {
  order_id varchar
  customer_id varchar
  order_status varchar
  order_purchase_timestamp varchar
  order_approved_at varchar
  order_delivered_carrier_date varchar
  order_delivered_customer_date varchar
  order_estimated_delivery_date varchar
}

Table items {
  order_id varchar
  order_item_id int
  product_id varchar
  seller_id varchar
  shipping_limit_date varchar
  price decimal
  freight_value decimal
}

Table payments {
  order_id varchar
  payment_sequential int
  payment_type varchar
  payment_installments int
  payment_value decimal
}

Table reviews {
  review_id varchar
  order_id varchar
  review_score int
  review_comment_title varchar
  review_c

## **Data Cleaning**

➤ Copy raw DataFrames into a new working dictionary to preserve the original data before cleaning.

In [27]:
# Create a new dictionary with copies of all raw DataFrames
def copy_raw_dataframes(raw_dict, exclude=None):
    """
    Creates copies of raw DataFrames to preserve the original data before any cleaning steps.

    Parameters:
        raw_dict (dict): Dictionary containing raw DataFrames.
        exclude (list): List of table names to exclude from copying.

    Returns:
        dict: A new dictionary with copies of the DataFrames.
    """
    exclude = exclude or []
    copy_dict = {}

    for name, df in raw_dict.items():
        if name in exclude:
            continue
        copy_dict[name] = df.copy()

    return copy_dict


dataframes = copy_raw_dataframes(dataframes_raw, exclude=["geolocation"])  # exclude 'geolocation', which is not used in the analysis

➤ Dropping unnecessary columns.

In [28]:
# Define columns to be dropped from specific DataFrames based on project scope
dropping_columns_dict = {
    "customers": "customer_zip_code_prefix",
    "reviews": ["review_comment_title", "review_comment_message"],
    "products": [
        "product_weight_g",
        "product_length_cm",
        "product_height_cm",
        "product_width_cm",
    ],
    "sellers": "seller_zip_code_prefix",
}

In [ ]:
# Drop predefined columns from each DataFrame based on a dictionary mapping
def drop_columns(df_dict, drop_dict):
    """
    Drops specified columns from DataFrames within a dictionary.

    Parameters:
        df_dict (dict): Dictionary of DataFrames to be modified.
        drop_dict (dict): Dictionary mapping DataFrame names to the columns
                          that should be dropped (single string or list of strings).

    Modifies:
        The DataFrames in df_dict are updated in-place with the specified columns removed.
    """
    for name, df in df_dict.items():
        if name not in drop_dict:
            continue

        # Ensure drop_dict[name] is a list, even if a single column is provided
        cols_to_drop = drop_dict[name]
        if isinstance(cols_to_drop, str):
            cols_to_drop = [cols_to_drop]

        # Drop columns that exist in the current DataFrame
        for col in cols_to_drop:
            if col in df.columns:
                df.drop(columns=col, inplace=True)

drop_columns(dataframes, dropping_columns_dict)